# Fusion Conv-BN et RepVGG

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAvgPool2D, Flatten, ReLU, Softmax, Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Add
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import backend

In [5]:
img_shape = 32, 32, 3

## Etude des poids des Conv $3 \times 3$ et des BatchNorm

Regardons comment s'articulent les poids dans les couches de convolutions et de batchnormalisation.

### Initialisation d'un modèle

In [6]:
input = Input(img_shape)
x= Conv2D(filters = 16, kernel_size=3, padding='same', use_bias=True, kernel_initializer='he_uniform', name='testing_conv_init')(input)
x= BatchNormalization(name=f'testing_bn_init')(x)
model = Model(input, x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
testing_conv_init (Conv2D)   (None, 32, 32, 16)        448       
_________________________________________________________________
testing_bn_init (BatchNormal (None, 32, 32, 16)        64        
Total params: 512
Trainable params: 480
Non-trainable params: 32
_________________________________________________________________


### Etude de la couche convolutive

In [7]:
weights_conv = model.get_layer("testing_conv_init").get_weights()
weights_conv

[array([[[[-0.09736294,  0.01110744,  0.38817808,  0.02365676,
            0.37265095,  0.22067323,  0.44893858, -0.4457162 ,
            0.2723634 ,  0.21101996, -0.42767352,  0.39105812,
           -0.38641602, -0.39619464, -0.12856498,  0.00230291],
          [-0.15622476,  0.08576027, -0.39533868,  0.14336786,
            0.09569708,  0.05594608,  0.05045763, -0.15595007,
           -0.05612397, -0.19001147,  0.2724487 ,  0.3459774 ,
            0.01586419,  0.08192965,  0.32559904,  0.04557905],
          [-0.37503266,  0.05977681, -0.05365878,  0.34279034,
           -0.22699383, -0.20862746,  0.13931164, -0.20776296,
            0.12117836,  0.06501201, -0.28448707,  0.2668244 ,
            0.2704514 , -0.34608564, -0.35193035, -0.3525829 ]],
 
         [[ 0.11737838,  0.14824674, -0.00563487,  0.3061553 ,
            0.01097953,  0.23561516, -0.4535907 , -0.4175086 ,
            0.4607807 , -0.37212858, -0.30806714,  0.14160755,
            0.24837866,  0.12601265,  0.2622976 ,

Les poids forment une liste de deux élements : les poids des noyaux de convolutions et les biais. La méthode d'initalisation utilisée ici est `he_uniform`, développée dans l'article [Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification](https://arxiv.org/abs/1502.01852)

In [8]:
type(weights_conv)

list

In [9]:
len(weights_conv)

2

Les poids dans une couche convolutive sont une liste de deux éléments : 
- `weights[0]` correspond aux poids des noyaux de convolution,
- `weights[1]` correspond aux biais.

In [10]:
type(weights_conv[0])

numpy.ndarray

In [11]:
weights_conv[0].shape 

(3, 3, 3, 16)

Les axes du tenseur de poids suivent les dimensions suivantes :

- kernel_size1 : hauteur du kernel,
- kernel_size2 : largeur du kernel,
- channels_in : nombre des feature maps en entrée, 
- channels_out : nombres de features maps (filters) en sortie.

`channels_out` est définie dans la couche convolutive via le paramètres `filters`, alors que la valeur `channels_in` est elle directement déterminée par le tenseur en entrée. C'est une différence de TensorFlow par rapport à Pytorch où `channels_in` et `channels_out` sont tous les deux des paramètres des couches convolutives.

Ainsi, si l'on veut voir les poids du noyau de convolution par rapport au canal $0$ en la feature map de sortie $5$, on les obtient en regardant :

In [12]:
weights_conv[0][:,:,0,5]

array([[ 0.22067323,  0.23561516, -0.13399044],
       [ 0.31607136,  0.23946908, -0.14616191],
       [ 0.18188533,  0.41285995,  0.1684458 ]], dtype=float32)

Par défaut, les biais des couches de convolutions sont tous initialisés à zéro.

In [13]:
weights_conv[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

### Etude de la batchnorm

In [14]:
weights_bn = model.get_layer('testing_bn_init').get_weights()
weights_bn

[array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=float32)]

In [15]:
type(weights_bn)

list

In [16]:
len(weights_bn)

4

Dans une couche de Batchnormalization, on a 4 types de poids.

- Les deux paramètres de scaling $\gamma$ et de biais $\beta$.
- Les deux paramètres correspondant à la moyenne $\mu$ et la variance $\sigma$.

Tous ces paramètres ne sont pas entraînables, comme on peut le voir dans la liste suivante.

In [ ]:
[(var.name, var.trainable) for var in model.get_layer('testing_bn_init').variables]

In [18]:
backend.shape(model.get_layer('testing_bn_init').get_weights())

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 4, 16], dtype=int32)>

Les $4$ paramètres sont tous des vecteurs de dimension $16$, ce qui correspond au nombre de feature maps en sortie de la couche convolutive. 

## Fusion d'une Convolution et d'une batchnorm

La fusion d'une couche de convolution avec une couche de batchnorm ressort les poids et biais d'une nouvelle couche de convolution avec les noyaux de convolutions de même dimension.

Etant donné le tenseur $W$ de poids des noyaux de convolution d'une couche convolutive et le tenseur de $4$ paramètres $B=(\gamma, \beta, \mu, \sigma)$ d'une couche de batchnormalization, on obtient les nouveaux poids et poids de la nouvelle couche convolutive via les formules suivantes.


$$
\widehat{W}_{:,:,:,j} := \frac{\gamma_{j} \cdot W_{:,:,:,j}}{\sqrt{\sigma_{j} + \epsilon}}
$$


$$
b_{j} = \beta_{j} - \frac{\mu_{j}\cdot\gamma_{j}}{\sqrt{\sigma_{j} + \epsilon}} 
$$

On remarque ici que le biais de la nouvelle couche de convolution ne dépend que des paramètres de la couche de batchnorm. **Ce qui est cohérent avec la pratique de ne jamais mettre de biais dans une couche de convolution lorsqu'elle est suivie par une couche de batchnorm**.


**Remarque** : le $\epsilon$ présent ici est pour s'assurer que l'on ne divise jamais pas zéro, dans la pratique il est fixé à $0,001$.

Ce qui nous donne, dans la pratique la fonction suivante.

In [19]:
# https://scortex.io/batch-norm-folding-an-easy-way-to-improve-your-network-speed/
# https://github.com/DingXiaoH/RepVGG/blob/4da799e33c890c624bfb484b2c35abafd327ba40/repvgg.py#L68

def fuse_bn_conv(weights_conv, weights_bn, eps=0.001):
    gamma = np.reshape(weights_bn[0], (1,1,1,weights_bn[0].shape[0]))
    beta = weights_bn[1]
    mean = weights_bn[2]
    variance = np.reshape(weights_bn[3], (1,1,1,weights_bn[3].shape[0]))

    new_weights = (weights_conv[0]*gamma) / np.sqrt(variance + eps)
    new_bias = beta - mean*gamma/np.sqrt(variance+eps)

    new_bias = np.reshape(new_bias, weights_bn[3].shape[0])

    return new_weights, new_bias

# In the code above, the reshaping is necessary to prevent a mistake if the dimension of the output O was the same as the dimension of the input I. 

# def get_equivalent_kernel_bias(self):
#    kernel3x3, bias3x3 = self._fuse_bn_tensor(self.rbr_dense)
#    kernel1x1, bias1x1 = self._fuse_bn_tensor(self.rbr_1x1)
#    kernelid, biasid = self._fuse_bn_tensor(self.rbr_identity)
#    return kernel3x3 + self._pad_1x1_to_3x3_tensor(kernel1x1) + kernelid, bias3x3 + bias1x1 + biasid

Détaillons la fonction ci dessus.

### Nouveau tenseur de poids

Discutons premièrement de la formulation du nouveau tenseur de poids, et voyons pourquoi on modifie la forme de vecteurs $\gamma$ et $\sigma$.

$W_{:,:,:,j}$ correspond dans la formule au noyau de convolution complet de la $j$-ième feature map de sortie.

In [20]:
weights_conv = model.get_layer("testing_conv_init").get_weights()
weights_conv[0].shape

(3, 3, 3, 16)

On a $16$ noyaux de convolution, chacun de dimensions $(3,3,3)$. Par exemple, pour $j=1$.

In [21]:
weights_conv[0][:,:,:,1].shape

(3, 3, 3)

Les vecteur $\gamma$ et $\sigma$ étant des vecteurs de dimension $16$, on va les "transformer en tenseur" de dimensions $(1,1,1,16)$ pour bien faire correspondre le produit suivant chaque axe.

In [22]:
variance = np.reshape(weights_bn[3], (1,1,1,weights_bn[3].shape[0]))
variance.shape

(1, 1, 1, 16)

In [23]:
gamma = np.reshape(weights_bn[0], (1,1,1,weights_bn[0].shape[0]))
gamma.shape

(1, 1, 1, 16)

![screen](images/fuse_conv_bn.svg)

Au final, la formule

```python
new_weights = (weights_conv[0]*gamma) / np.sqrt(variance + eps)
```

résume tout cela, tous les tenseurs ayant le nombre d'axes, les opérations sont vectorisées et se font axe par axe.

### Nouveau tenseur de biais

Le opérations de `reshape` n'ont pas ajouter de nouveaux scalaires, juste des axes, le calcul du biais se fait alors élément par élément pour tout $j$.

### Vérification via les développements limités

Créons un tenseur de poids $W$ repéresentatif du noyau d'une convolution et un tenseur de poids $B=(\gamma, \beta, \mu, \sigma)$ représentatif des coefficients d'une batchnormalization.

Pour vérifier si tout marche bien, fixons volontairement le tenseur poids comme un tenseur de dimensions $(3,3,4,5)$, la dimension du noyau est toujours fixé à $(3,3)$ dans RepVGG, seules les dimensions `channels_in` et `channels_out` peuvent changer.

Tous les coefficients du tenseur de poids seront fixés à $1$.

In [24]:
conv_weights = np.ones(3*3*4*5).reshape((3,3,4,5))
conv_weights.shape

(3, 3, 4, 5)

La dimension `channels_out` ayant été fixée à $5$, les vecteurs de la batchnormalization seront tous des vecteurs de dimension $5$. Fixons les coefficients suivants.

In [25]:
def batchnorm_variables(gamma_coef: float, beta_coef: float, mu_coef: float, sigma_coef: float, channels: int):
    gamma = gamma_coef*np.ones(channels)
    beta = beta_coef*np.ones(channels)
    mu = mu_coef*np.ones(channels)
    sigma = sigma_coef*np.ones(channels)
    
    return [gamma, beta, mu, sigma]

In [26]:
conv, bn = fuse_bn_conv([conv_weights], batchnorm_variables(1,2,1,4,5))

Par définition, le nouveau tenseur de poids $\widehat{W}$ de la convolution résultant de la fusion de l'ancienne convolution et de la batchnorm est donné par formule suivante.

$$
\widehat{W}_{:,:,:,j} := \frac{\gamma_{j} \cdot W_{:,:,:,j}}{\sqrt{\sigma_{j} + \epsilon}}
$$

De façon générale, pour $\gamma_{j}, \sigma_{j}$, on a le développement limité suivant.

$$
\widehat{W}_{:,:,:,j} := \frac{\gamma_{j} \cdot W_{:,:,:,j}}{\sqrt{\sigma_{j} + \epsilon}} = \frac{\gamma_{j}}{\sqrt{\sigma_{j}}}\left[1- \frac{1}{2\sigma_{j}}\epsilon + o(\epsilon^{2})\right]W_{:,:,:,j} 
$$

Dans notre cas, $\forall j, \gamma_{j} = 1, \sigma_{j} = 4$ d'où

$$
\widehat{W}_{:,:,:,j} := \frac{W_{:,:,:,j}}{\sqrt{4 + \epsilon}} = \left[\frac{1}{2}- \frac{1}{16}\epsilon + o(\epsilon^{2})\right]W_{:,:,:,j} \simeq \left[\frac{1}{2}- \frac{1}{16}\epsilon\right]W_{:,:,:,j}
$$

In [27]:
def compute_scaling_weight_factor(gamma, sigma):
    return gamma/np.sqrt(sigma)*(1-0.001/(2*sigma))

In [28]:
scale = compute_scaling_weight_factor(1,4)
scale

0.4999375

In [29]:
conv[:,:,:,4]

array([[[0.49993751, 0.49993751, 0.49993751, 0.49993751],
        [0.49993751, 0.49993751, 0.49993751, 0.49993751],
        [0.49993751, 0.49993751, 0.49993751, 0.49993751]],

       [[0.49993751, 0.49993751, 0.49993751, 0.49993751],
        [0.49993751, 0.49993751, 0.49993751, 0.49993751],
        [0.49993751, 0.49993751, 0.49993751, 0.49993751]],

       [[0.49993751, 0.49993751, 0.49993751, 0.49993751],
        [0.49993751, 0.49993751, 0.49993751, 0.49993751],
        [0.49993751, 0.49993751, 0.49993751, 0.49993751]]])

Ce qui correspond bien à l'approximation obtenue par développement limité. On peut par exemple vérifier si $\widehat{W}$ est approximativement égal à `conv` à $10^{-3}$ avec la commande `np.isclose`.

In [30]:
conv_weights_real = scale*np.ones(3*3*4*5).reshape((3,3,4,5))
conv_weights_real[:,:,:,0]

array([[[0.4999375, 0.4999375, 0.4999375, 0.4999375],
        [0.4999375, 0.4999375, 0.4999375, 0.4999375],
        [0.4999375, 0.4999375, 0.4999375, 0.4999375]],

       [[0.4999375, 0.4999375, 0.4999375, 0.4999375],
        [0.4999375, 0.4999375, 0.4999375, 0.4999375],
        [0.4999375, 0.4999375, 0.4999375, 0.4999375]],

       [[0.4999375, 0.4999375, 0.4999375, 0.4999375],
        [0.4999375, 0.4999375, 0.4999375, 0.4999375],
        [0.4999375, 0.4999375, 0.4999375, 0.4999375]]])

Si `np.mean(...)` $< 1$ alors le calcul est faux.

In [31]:
np.mean(np.isclose(conv, conv_weights_real, rtol=1e-3))

1.0

Pour le biais, on a la formule suivante.

$$
b_{j} = \beta_{j} - \frac{\mu_{j}\cdot\gamma_{j}}{\sqrt{\sigma_{j} + \epsilon}} = \beta_{j} - \frac{\mu_{j}\cdot\gamma_{j}}{\sqrt{\sigma_{j}}}\left[1- \frac{1}{2\sigma_{j}}\epsilon + o(\epsilon^{2})\right]
$$

dans notre cas, on a :

- $\beta_{j} = 2$,
- $\gamma_{j} = 1$,
- $\mu_{j} = 1$,
- $\sigma_{j} = 4$.

$$
b_{j} = 2 - \frac{1}{2}\left[1- \frac{1}{8}\epsilon + o(\epsilon^{2})\right] \simeq 2 - \frac{1}{2} - \frac{1}{16}\epsilon
$$


In [2]:
def compute_scaling_bias_factor(gamma, beta, mu, sigma):
    a = (mu*gamma)/np.sqrt(sigma)
    b = 1 - 0.001/(2*sigma)
    
    return beta-a*b

In [ ]:
bias_scale = compute_scaling_bias_factor(1,2,1,4)
bias_scale

In [34]:
bn

array([1.50006249, 1.50006249, 1.50006249, 1.50006249, 1.50006249])

In [35]:
bn_real = bias_scale*np.ones(5)
np.mean(np.isclose(bn_real, bn, rtol=1e-3))

1.0

## RepVGG

![screen](./images/repvgg.svg)

![screen](./images/repvgg2.svg)

Les couches convolutives dans RepVGG n'ayant que des noyaux $3\times3$ ou $1\times1$, on ne se préoccupe que de cela dans la suite.

## Fusion d'une Conv $3\times3$ avec une batchnorm puis transfert de poids

Créons un modèle simple : une couche convolutive suivi d'une couche de batchnormalisation, pour simplifier on ne condière aucune couche d'activation (qui de toute façon ne rentre pas en jeu). Nous allons :

1. Fusionner les deux couches pour créer un nouveau tenseur (poids, biais)
2. Transférer ce nouveau tensor dans un modèle plus simple `model_after_fusion`.

**Remarque** : la convolution dans `model_after_fusion` utilise elle bien un biais (`use_bias = True`).

In [36]:
input = Input(img_shape)
x= Conv2D(filters = 16, kernel_size=3, padding='same', use_bias=False, kernel_initializer='he_uniform', name='conv')(input)
x= BatchNormalization(name='bn')(x)
model_before_fusion = Model(input, x)
model_before_fusion.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv (Conv2D)                (None, 32, 32, 16)        432       
_________________________________________________________________
bn (BatchNormalization)      (None, 32, 32, 16)        64        
Total params: 496
Trainable params: 464
Non-trainable params: 32
_________________________________________________________________


In [37]:
input = Input(img_shape)
x= Conv2D(filters = 16, kernel_size=3, padding='same', use_bias=True, kernel_initializer='he_normal', name='conv')(input)
model_after_fusion = Model(input, x)
model_after_fusion.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv (Conv2D)                (None, 32, 32, 16)        448       
Total params: 448
Trainable params: 448
Non-trainable params: 0
_________________________________________________________________


In [38]:
weights_1 = model_before_fusion.get_layer('conv').get_weights()[0]
weights_2 = model_after_fusion.get_layer('conv').get_weights()[0]

In [39]:
np.mean(weights_1-weights_2)

0.0023266133

In [40]:
conv = model_before_fusion.get_layer("conv")
bn = model_before_fusion.get_layer("bn")
  
conv_weights, conv_biases = fuse_bn_conv(conv.get_weights(), bn.get_weights())
model_after_fusion.get_layer(f"conv").set_weights([conv_weights, conv_biases])

Vérifions que la mise en place des nouveaux poids s'est bien passée, ie que l'opération `set_weights()` n'a rien ajouté de supplémtentaire. Si tout se passe bien, `np.mean` ne devrait renvoyer que des `1.0`.

In [41]:
w0, b0 = fuse_bn_conv(model_before_fusion.get_layer("conv").get_weights(), model_before_fusion.get_layer("bn").get_weights())

w1, b1 = model_after_fusion.get_layer("conv").get_weights()

In [42]:
np.mean(w0 == w1)

1.0

In [43]:
np.mean(b0 == b1)

1.0

Donc tout s'est bien passé. Reste maintenant à généraliser cette transformation.

L'idée de RepVGG est d'utiliser une architecture à la ResNet pour l'entraînement, avec des skips connections, puis lors du déploiement du modèle de reparamétrer les skips connections via des fusions Conv-BN afin de plus avoir qu'une architecture linéaire à la VGG, beaucoup plus rapide en inférence qu'une architecture à la ResNet.

En plus de fusionner des $\mathrm{Conv} 3 \times 3$ avec des $\mathrm{BN}$, il est aussi nécessaire de savoir faire les opérations suivantes.

1. Convertir une $\mathrm{Conv} 1 \times 1$ en $\mathrm{Conv} 3 \times 3$ puis la fusionner avec la $\mathrm{BN}$ correspondante.
2. Convertir une $\mathrm{id}$ en $\mathrm{Conv} 3 \times 3$ puis la fusionner avec la $\mathrm{BN}$ correspondante.

## Conversion d'une Conv $1 \times 1$ en $3 \times 3$ puis fusion avec la batchnorm.

Pour convertir une conv 1x1 en conv 3x3 les nombres de canaux en entrée et en sortie importe peu, ce qu'il faut c'est modifier la dimension des noyaux de convolutions pour passer d'une dimension 1x1 à 3x3, et pour cela on utilise un padding.

In [44]:
input = Input(img_shape)
x= Conv2D(filters = 16, kernel_size=1, padding='same', use_bias=False, kernel_initializer='he_uniform', name='conv')(input)
x= BatchNormalization(name='bn')(x)
model_before_fusion_conv1 = Model(input, x)
model_before_fusion_conv1.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv (Conv2D)                (None, 32, 32, 16)        48        
_________________________________________________________________
bn (BatchNormalization)      (None, 32, 32, 16)        64        
Total params: 112
Trainable params: 80
Non-trainable params: 32
_________________________________________________________________


In [45]:
input = Input(img_shape)
x= Conv2D(filters = 16, kernel_size=3, padding='same', use_bias=True, kernel_initializer='he_normal', name='conv')(input)
model_after_fusion_conv1 = Model(input, x)
model_after_fusion_conv1.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv (Conv2D)                (None, 32, 32, 16)        448       
Total params: 448
Trainable params: 448
Non-trainable params: 0
_________________________________________________________________


In [46]:
weights_conv1 = model_before_fusion_conv1.get_layer('conv')
weights_bn1 = model_before_fusion_conv1.get_layer('bn')

In [47]:
weights_conv1.get_weights()[0].shape

(1, 1, 3, 16)

La première chose à faire, c'est de transformer les noyaux de convolution $1\times1$ en des noyaux de convolution $3\times3$. Pour faire cela, on utilise la notion de "padding", déjà utilisée dans le cas des convolutions.

In [48]:
weights_conv1.get_weights()[0][:,:,1,1]

array([[-1.227452]], dtype=float32)

On a deux fonctions possibles pour faire ça. On peut utiliser soit la fonction de tensorflow.

```python
padded_conv1 = tf.pad(weights_conv1.get_weights()[0], [[1,1], [1, 1], [0,0], [0,0]], "CONSTANT")
```

Soit la fonction de numpy.

```python
padded_conv1 = np.pad(weights_conv1.get_weights()[0], pad_width=[[1,1], [1, 1], [0,0], [0,0]], mode='constant', constant_values=0)
```

Dans les deux cas, on a un paramètre donnant la taille du padding : `[[1,1], [1, 1], [0,0], [0,0]]`, c'est une liste de longueur le nombre d'axes du tenseur que l'on souhaite modifier, chaque élément de la liste nous dit de combien on doit agrandir au début et à la fin.

`[[1,1], [1, 1], [0,0], [0,0]] = [[pad_avant_axe1, pad_arrière_axe1], [pad_avant_axe2, pad_arrière_axe2], [pad_avant_axe3, pad_arrière_axe3], [pad_avant_axe4, pad_arrière_axe4]]`

Le dernier paramètre nous dit quoi rajouter aux endroits où l'on a agrandi, ici des constantes : la valeur $0$.

In [49]:
padded_conv1_tf = tf.pad(weights_conv1.get_weights()[0], [[1,1], [1, 1], [0,0], [0,0]], "CONSTANT")
padded_conv1_np = np.pad(weights_conv1.get_weights()[0], pad_width=[[1,1], [1, 1], [0,0], [0,0]], mode='constant', constant_values=0)

Les deux fonctions donnent le même résultat.

In [50]:
np.mean(padded_conv1_tf.numpy()==padded_conv1_np)==1

True

Comme la fonction `set_weights()` demande d'utiliser des `np.array`, on va utiliser la fonction de numpy.

In [51]:
def pad_size_one_kernel(conv_weights):    
    return np.pad(conv_weights[0], pad_width=[[1,1], [1, 1], [0,0], [0,0]], mode='constant', constant_values=0)

In [52]:
padded_weights_conv1 = pad_size_one_kernel(weights_conv1.get_weights())
padded_weights_conv1.shape

(3, 3, 3, 16)

### Vérification

On a transformé tous les noyaux de convolutions $1\times1$ en noyaux $3\times3$, chacun des `padded_weights_conv1[:,:,i,j]` pour $0 \leq i \leq 2$ et $0 \leq j \leq 15$ doit être une matrice $3\times3$ où tous les éléments sont nuls sauf possiblement celui du milieu.

In [53]:
def test_padded_kernel_conv(padded_kernel):
    for i in range(3):
        for j in range(16):
            print(f'Matrix of size 3x3 : {padded_kernel[:,:,i,j].shape == (3,3)}')
            squared_sum = 0
            for k in range(3):
                for l in range(3):
                    if k != 1 and l != 1:
                        squared_sum += padded_kernel[:,:,i,j][k,l]**2
            print(f'Squared sum is : {squared_sum}')

In [54]:
test_padded_kernel_conv(padded_weights_conv1)

Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of siz

In [55]:
dummy_conv = np.ones(1*1*3*16).reshape((1,1,3,16))

In [56]:
padded_dummy_conv=pad_size_one_kernel([dummy_conv])
test_padded_kernel_conv(padded_dummy_conv)

Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of size 3x3 : True
Squared sum is : 0.0
Matrix of siz

Comme précédemment, on vérifie via les développements limités que ça fonctionne.

In [57]:
conv, bn = fuse_bn_conv([padded_dummy_conv], batchnorm_variables(1,2,1,4,16))

In [58]:
scale = compute_scaling_weight_factor(1,4)
scale

0.4999375

In [59]:
np.mean(np.isclose(conv, scale*padded_dummy_conv, rtol=1e-3))

1.0

In [60]:
bias_scale = compute_scaling_bias_factor(1,2,1,4)
bias_scale

1.5000625

In [61]:
bn_real = bias_scale*np.ones(16)
np.mean(np.isclose(bn_real, bn, rtol=1e-3))

1.0

In [62]:
weights_conv1 = model_before_fusion_conv1.get_layer('conv')
weights_bn1 = model_before_fusion_conv1.get_layer('bn')

padded_weights_conv1 = pad_size_one_kernel(weights_conv1.get_weights())
conv_weights, conv_bias = fuse_bn_conv([padded_weights_conv1], weights_bn1.get_weights())

model_after_fusion_conv1.get_layer("conv").set_weights([conv_weights, conv_bias])

## Conversion d'une $\mathrm{id}$ en $\mathrm{Conv} 3 \times 3$ puis fusion avec la batchnorm.

Les branches id ne sont utilisées dans l'architecture de RepVGG que lorsque la conditions `channels_in` = `channels_out` est vérifiée, c'est à dire à l'intérieur de chaque stage entre 2 blocs convolutifs avec un stride de 2.

In [63]:
# Fixons le nombre de channels, peut importe le nombre.
channels = 4

An identity mapping can be viewed as a $1\times1$ conv with an identity matrix as the kernel.

In [64]:
def size_three_kernel_from_id(channels):
    kernel = np.ones(channels)
    kernel = np.diag(kernel)
    kernel = np.reshape(kernel, (1,1,channels,channels))
    kernel = np.pad(kernel, pad_width=[[1,1], [1, 1], [0,0], [0,0]], mode='constant', constant_values=0)

    return kernel

In [65]:
conv_from_id = size_three_kernel_from_id(4)
conv_from_id.shape

(3, 3, 4, 4)

In [66]:
def test_padded_kernel_from_id(padded_kernel):
    for i in range(3):
        for j in range(16):
            print(f'Matrix of size 3x3 : {padded_kernel[:,:,i,j].shape == (3,3)}')
            squared_sum = 0
            for k in range(3):
                for l in range(3):
                    if (k,l) != (1,1):
                        squared_sum += padded_kernel[:,:,i,j][k,l]**2
                    else:
                        print(f'Middle element is 1 : {padded_kernel[:,:,i,j][k,l]==1}')
            print(f'Squared sum is : {squared_sum}')

In [67]:
test_padded_kernel_from_id(conv)

Matrix of size 3x3 : True
Middle element is 1 : False
Squared sum is : 0.0
Matrix of size 3x3 : True
Middle element is 1 : False
Squared sum is : 0.0
Matrix of size 3x3 : True
Middle element is 1 : False
Squared sum is : 0.0
Matrix of size 3x3 : True
Middle element is 1 : False
Squared sum is : 0.0
Matrix of size 3x3 : True
Middle element is 1 : False
Squared sum is : 0.0
Matrix of size 3x3 : True
Middle element is 1 : False
Squared sum is : 0.0
Matrix of size 3x3 : True
Middle element is 1 : False
Squared sum is : 0.0
Matrix of size 3x3 : True
Middle element is 1 : False
Squared sum is : 0.0
Matrix of size 3x3 : True
Middle element is 1 : False
Squared sum is : 0.0
Matrix of size 3x3 : True
Middle element is 1 : False
Squared sum is : 0.0
Matrix of size 3x3 : True
Middle element is 1 : False
Squared sum is : 0.0
Matrix of size 3x3 : True
Middle element is 1 : False
Squared sum is : 0.0
Matrix of size 3x3 : True
Middle element is 1 : False
Squared sum is : 0.0
Matrix of size 3x3 : True

In [68]:
conv, bn = fuse_bn_conv([conv_from_id], batchnorm_variables(1,2,1,4,channels))

In [69]:
scale = compute_scaling_weight_factor(1,4)
scale

0.4999375

In [70]:
np.mean(np.isclose(conv, scale*conv_from_id, rtol=1e-3))

1.0

In [71]:
bias_scale = compute_scaling_bias_factor(1,2,1,4)
bias_scale

1.5000625

In [72]:
bn_real = bias_scale*np.ones(channels)
np.mean(np.isclose(bn_real, bn, rtol=1e-3))

1.0

### Vérification

## Test grandeur réelle

In [77]:
def repvgg_block(tensor, filters, num_layer):
    
    # main stream
    x = Conv2D(
        filters=filters,
        kernel_size=(3,3),
        strides=(2,2),
        padding="same",
        kernel_initializer="he_normal",
        use_bias=False,
        name=f'block_{num_layer}_conv_main'
    )(tensor)
    x = BatchNormalization(name=f'block_{num_layer}_bn_main')(x)
    
    # conv1x1 stream
    
    y = Conv2D(
        filters=filters,
        kernel_size=(1,1),
        strides=(2,2),
        padding="same",
        kernel_initializer="he_normal",
        use_bias=False,
        name=f'block_{num_layer}_conv_alt'
    )(tensor)
    y = BatchNormalization(name=f'block_{num_layer}_bn_alt')(y)
    
    z = Add()([x,y])
    
    return z
    

In [78]:
def repvgg_block_with_id(tensor, filters, num_layer):

    # main stream
    x = Conv2D(
        filters=filters,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding="same",
        kernel_initializer="he_normal",
        use_bias=False,
        name=f"block_{num_layer}_conv_main",
    )(tensor)
    x = BatchNormalization(name=f"block_{num_layer}_bn_main")(x)

    # conv1x1 stream

    y = Conv2D(
        filters=filters,
        kernel_size=(1, 1),
        strides=(1, 1),
        padding="same",
        kernel_initializer="he_normal",
        use_bias=False,
        name=f"block_{num_layer}_conv_alt",
    )(tensor)
    y = BatchNormalization(name=f"block_{num_layer}_bn_alt")(y)

    # id_conv branch
    z = BatchNormalization(name=f"block_{num_layer}_bn_id")(tensor)

    return Add()([x, y, z])

In [114]:
def get_model(img_shape):

    input = Input(img_shape)
    
    x = repvgg_block(input, filters=64, num_layer=0)
    x = ReLU()(x)
    x = repvgg_block(x, filters=64, num_layer=1)
    x = ReLU()(x)
    x = repvgg_block_with_id(x, filters=64, num_layer=2)
    x = ReLU()(x)
    x = Flatten()(x)
    x = Dense(10, name='dense')(x)
    x = Softmax()(x)
    model = Model(input, x)
    return model

def get_inference_model(img_shape):
    input = Input(img_shape)
    
    x = Conv2D(filters=64, kernel_size=(3,3), strides=(2,2), padding='same', name='conv_0')(input)
    x = ReLU()(x)
    x = Conv2D(filters=64, kernel_size=(3,3), strides=(2,2), padding='same', name='conv_1')(x)
    x = ReLU()(x)
    x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', name='conv_2')(x)
    x = ReLU()(x)
    x = Flatten()(x)
    x = Dense(10, name='dense')(x)
    x = Softmax()(x)
    model = Model(input, x)
    return model

In [115]:
training_model = get_model([32,32,3])
training_model.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
block_0_conv_main (Conv2D)      (None, 16, 16, 64)   1728        input_14[0][0]                   
__________________________________________________________________________________________________
block_0_conv_alt (Conv2D)       (None, 16, 16, 64)   192         input_14[0][0]                   
__________________________________________________________________________________________________
block_0_bn_main (BatchNormaliza (None, 16, 16, 64)   256         block_0_conv_main[0][0]          
___________________________________________________________________________________________

In [116]:
inference_model = get_inference_model([32,32,3])
inference_model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv_0 (Conv2D)              (None, 16, 16, 64)        1792      
_________________________________________________________________
re_lu_23 (ReLU)              (None, 16, 16, 64)        0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 8, 8, 64)          36928     
_________________________________________________________________
re_lu_24 (ReLU)              (None, 8, 8, 64)          0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 8, 8, 64)          36928     
_________________________________________________________________
re_lu_25 (ReLU)              (None, 8, 8, 64)          0  

In [117]:
def from_repvgg_to_vgg(training_model, inference_model, depth):
    model = training_model
    inference_model = inference_model
    
    for i in range(depth):
        print(f"Fusion Conv-BN from main branch at depth {i}")
        conv_main = model.get_layer(f"block_{i}_conv_main")
        bn_main = model.get_layer(f"block_{i}_bn_main")

        conv_weights_main, conv_biases_main = fuse_bn_conv(
            conv_main.get_weights(), bn_main.get_weights()
        )

        print(f"Fusion Conv-BN from alt branch at depth {i}")
        conv_alt_one_by_one = model.get_layer(f"block_{i}_conv_alt")
        bn_alt = model.get_layer(f"block_{i}_bn_alt")

        conv_alt = pad_size_one_kernel(conv_alt_one_by_one.get_weights())

        conv_weights_alt, conv_biases_alt = fuse_bn_conv([conv_alt], bn_alt.get_weights())
        
        if i==3:
            print(f"Fusion Conv-BN from id branch at depth {i}")
            bn_id = model.get_layer(f"block_{i}_bn_id")
            channels = backend.int_shape(bn_id.get_weights()[0])[-1]

            conv_id = size_three_kernel_from_id(channels)
            conv_weights_id, conv_biases_id = fuse_bn_conv([conv_id], bn_id.get_weights())

            conv_weights = conv_weights_main + conv_weights_alt + conv_weights_id
            conv_biases = conv_biases_main + conv_biases_alt + conv_biases_id
        else:
            conv_weights = conv_weights_main + conv_weights_alt
            conv_biases = conv_biases_main + conv_biases_alt
            
           
        print(f"Setting weights on inference model at depth {i}")
        inference_model.get_layer(f"conv_{i}").set_weights([conv_weights, conv_biases])

    dense_weights = model.get_layer(f"dense").get_weights()
    inference_model.get_layer(f"dense").set_weights(dense_weights)
    
    return inference_model

In [118]:
from tensorflow.keras import datasets
from sklearn.model_selection import train_test_split

(X_train,y_train), (X_test,y_test)  = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
X_train, X_test = X_train / 255.0, X_test / 255.0

X_train = X_train.reshape(-1, 32, 32, 3).astype('float32')
X_test = X_test.reshape(-1, 32, 32, 3).astype('float32')


X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, random_state=42)

y_train_oh = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test_oh = tf.keras.utils.to_categorical(y_test, num_classes=10)
y_valid_oh = tf.keras.utils.to_categorical(y_valid, num_classes=10)

In [119]:
training_model.compile(loss = 'categorical_crossentropy',
             optimizer=tf.keras.optimizers.Adam(lr=0.001),
             metrics=['accuracy'])

training_model.fit(X_train, y_train_oh,
                     epochs = 200,
                     batch_size=128,
                     validation_data=(X_valid, y_valid_oh))

Epoch 1/200
293/293 [==============================] - 2s 4ms/step - loss: 1.9390 - accuracy: 0.3690 - val_loss: 1.4326 - val_accuracy: 0.4882
Epoch 2/200
293/293 [==============================] - 1s 3ms/step - loss: 1.1761 - accuracy: 0.5805 - val_loss: 1.2776 - val_accuracy: 0.5559
Epoch 3/200
293/293 [==============================] - 1s 3ms/step - loss: 0.9771 - accuracy: 0.6608 - val_loss: 1.2784 - val_accuracy: 0.5692
Epoch 4/200
293/293 [==============================] - 1s 3ms/step - loss: 0.8457 - accuracy: 0.7047 - val_loss: 1.1538 - val_accuracy: 0.6070
Epoch 5/200
293/293 [==============================] - 1s 3ms/step - loss: 0.7411 - accuracy: 0.7415 - val_loss: 1.1523 - val_accuracy: 0.6082
Epoch 6/200
293/293 [==============================] - 1s 3ms/step - loss: 0.6627 - accuracy: 0.7721 - val_loss: 1.2050 - val_accuracy: 0.6056
Epoch 7/200
293/293 [==============================] - 1s 3ms/step - loss: 0.5730 - accuracy: 0.8017 - val_loss: 1.3360 - val_accuracy: 0.5908

In [120]:
training_model.evaluate(X_test, y_test_oh)

313/313 [==============================] - 0s 1ms/step - loss: 4.0375 - accuracy: 0.6408


[4.037524223327637, 0.6407999992370605]

In [121]:
model = from_repvgg_to_vgg(training_model, inference_model, 3)
model.summary()

Fusion Conv-BN from main branch at depth 0
Fusion Conv-BN from alt branch at depth 0
Setting weights on inference model at depth 0
Fusion Conv-BN from main branch at depth 1
Fusion Conv-BN from alt branch at depth 1
Setting weights on inference model at depth 1
Fusion Conv-BN from main branch at depth 2
Fusion Conv-BN from alt branch at depth 2
Setting weights on inference model at depth 2
Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv_0 (Conv2D)              (None, 16, 16, 64)        1792      
_________________________________________________________________
re_lu_23 (ReLU)              (None, 16, 16, 64)        0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 8, 8, 64)          36928 

In [122]:
model.compile(loss = 'categorical_crossentropy',
             optimizer=tf.keras.optimizers.SGD(lr=2e-9),
             metrics=['accuracy'])


In [123]:
model.evaluate(X_test, y_test_oh)

313/313 [==============================] - 0s 1ms/step - loss: 7.8139 - accuracy: 0.3986


[7.867744445800781, 0.39640000462532043]